# Object를 추적하고, 그들의 좌표까지 기록할 수 있도록, 그리고 경기장 좌표까지

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi
import torch, sys, platform
print("Py:", sys.version)
print("Plat:", platform.platform())

Tue Sep 23 11:40:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
# 0) YOLO 설치
!pip -q install ultralytics==8.3.197 lap>=0.5.12

from ultralytics import YOLO
PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"  # 네 가중치
player_model = YOLO(PLAYER_MODEL)

In [4]:
!pip install hydra-core==1.3.2 omegaconf==2.3.0

!pip install --upgrade torchvision

!pip install decord

In [5]:
# ================================
# ✅ SAM2 패키지 경로 등록 & 설치
# ================================
import sys, os

# SAM2가 Drive에 저장돼 있다고 가정
SAM2_PATH = "/content/drive/MyDrive/sam2"
sys.path.append(SAM2_PATH)

# 패키지 설치 (Colab 런타임마다 필요)
!pip install -e /content/drive/MyDrive/sam2

# ================================
# ✅ 라이브러리 import
# ================================
import torch, os, cv2, numpy as np
from collections import deque  # ★ NEW
from ultralytics import YOLO
from sam2.build_sam import build_sam2_video_predictor

# ------------------------
# 준비
# ------------------------
def best_dtype():
    return torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
dtype = best_dtype()

# 성능 스위치 (가능한 경우) ★ NEW
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

os.chdir("/content/drive/MyDrive/sam2")
ckpt = "/content/drive/MyDrive/sam2/checkpoints/sam2.1_hiera_large.pt"
cfg  = "configs/sam2.1/sam2.1_hiera_l.yaml"
pred = build_sam2_video_predictor(cfg, ckpt, vos_optimized=False)

MP4 = "/content/drive/MyDrive/Little_kid_0912/Video_data/0903_day3.mp4"
with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    state = pred.init_state(MP4)

PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"
BALL_MODEL   = "/content/drive/MyDrive/Little_kid_0912/pose/ball/ball_800/train/weights/best.pt"
player_model = YOLO(PLAYER_MODEL)
ball_model   = YOLO(BALL_MODEL)

# 가능한 경우 모델 fuse 및 CUDA 이동 ★ NEW
try:
    player_model.fuse()
    ball_model.fuse()
except Exception:
    pass
player_model.to("cuda")
ball_model.to("cuda")

# ------------------------
# Util 함수
# ------------------------
def iou(boxA, boxB):
    """float 좌표에 맞춘 IOU ( +1 제거 )  ★ NEW"""
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    inter = max(0.0, xB - xA) * max(0.0, yB - yA)
    areaA = max(0.0, (boxA[2] - boxA[0])) * max(0.0, (boxA[3] - boxA[1]))
    areaB = max(0.0, (boxB[2] - boxB[0])) * max(0.0, (boxB[3] - boxB[1]))
    denom = areaA + areaB - inter + 1e-6
    return inter / denom

def box_center(box):
    x1,y1,x2,y2 = box
    return (0.5*(x1+x2), 0.5*(y1+y2))

id2color, ema_masks, class_of = {}, {}, {}
def color_for(oid: int):
    if oid not in id2color:
        rng = np.random.default_rng(oid + 12345)
        id2color[oid] = tuple(int(c) for c in rng.integers(60,255,size=3))
    return id2color[oid]

# ------------------------
# 초기 씨딩 (각 객체가 '처음' 보이는 프레임에서)
# ------------------------
dev = torch.device("cuda")
next_oid = 0
class_of = {}   # {oid: "player"|"ball"}
found_players = False
found_ball = False

# 스캔 설정
SCAN_MAX = 300          # 처음 N프레임까지만 스캔 (원하면 크게)
STEP     = 1            # 1프레임 단위로 스캔 (속도 필요시 2~3으로)
CONF_P   = 0.30         # 선수 conf
CONF_B   = 0.04         # 공 conf(더 낮춤) ★ NEW
IMGSZ_B  = 1536         # 공 입력 해상도 상향(감지력↑) ★ NEW

cap = cv2.VideoCapture(MP4)

def seed_object(fi, xyxy, label, use_box=True):
    """
    xyxy=[x1,y1,x2,y2]를 받아 SAM2에 씨딩.
    - use_box=True: box 프롬프트 + 중심점(point) 함께 전달(권장)
    - use_box=False: 중심점(point)만 전달
    """
    global next_oid
    x1, y1, x2, y2 = xyxy
    oid = next_oid

    with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
        if use_box:
            # ✅ 박스 + 포인트(같은 CUDA)
            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,   # ← 포인트/라벨 명시
                box=box_t,                 # ← 박스도 함께
                normalize_coords=True
            )
        else:
            # 포인트만
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,
                normalize_coords=True
            )

    class_of[oid] = label
    next_oid += 1
    return oid

seed_logs = []  # [(oid, label, fi)]

for fi in range(0, SCAN_MAX, STEP):
    ok, bgr = cap.read()
    if not ok:
        break

    # 초기 스캔: 선수 상위 N명만 씨딩(메모리/연산 낭비 방지) ★ NEW
    if not found_players:
        rp = player_model(bgr, conf=CONF_P, verbose=False)[0]
        if rp.boxes is not None and len(rp.boxes) > 0:
            confs = rp.boxes.conf.detach().float().cpu().numpy()
            order = np.argsort(-confs)[:10]  # 상위 10명만
            for idx in order:
                b = rp.boxes.xyxy[idx].detach().cpu().numpy().tolist()
                oid = seed_object(fi, b, "player", use_box=True)
                seed_logs.append((oid, "player", fi))
            found_players = True

    if not found_ball:
        # 공: 낮은 conf, 큰 imgsz, agnostic_nms 사용 ★ NEW
        rb = ball_model(bgr, conf=CONF_B, iou=0.4, imgsz=IMGSZ_B,
                        agnostic_nms=True, verbose=False)[0]
        if rb.boxes is not None and len(rb.boxes) > 0:
            # top-1이 아니라 가장 큰 bbox or conf 조합도 가능하지만 우선 conf 최대
            idx = int(rb.boxes.conf.argmax().detach().cpu())
            b = rb.boxes.xyxy[idx].detach().cpu().numpy().tolist()
            oid = seed_object(fi, b, "ball", use_box=True)
            seed_logs.append((oid, "ball", fi))
            found_ball = True

    # 둘 다 찾았으면 조기 종료
    if found_players and found_ball:
        break

cap.release()

# 요약 로그
n_players = sum(1 for _,lbl,_ in seed_logs if lbl=="player")
n_balls   = sum(1 for _,lbl,_ in seed_logs if lbl=="ball")
print("초기 씨딩 완료:",
      f"players={n_players}, ball={n_balls},",
      "details=", seed_logs)

def seed_from_yolo(res, label, fi=0):
    """현재는 사용 안 하지만 남겨둠"""
    global next_oid
    added = []
    if res.boxes is not None and len(res.boxes)>0:
        for b in res.boxes.xyxy.detach().cpu().numpy().tolist():
            x1,y1,x2,y2 = b
            xc,yc = (x1+x2)/2,(y1+y2)/2
            pts  = torch.tensor([[xc,yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],      device=dev, dtype=torch.long)
            oid  = next_oid
            with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
                pred.add_new_points_or_box(
                    state, fi, oid,
                    points=pts, labels=labs,
                    normalize_coords=True
                )
            class_of[oid] = label
            added.append((oid,[x1,y1,x2,y2]))
            next_oid += 1
    return added


# 비디오 해상도 먼저 가져오기
cap = cv2.VideoCapture(MP4)
W, H = int(cap.get(3)), int(cap.get(4))
cap.release()

# ===== [HOMO] 반코트(10x20m) 호모그래피 =====
# 픽셀 기준점: 시계방향으로 P1->P2->P3->P4 (동료가 준 값)
pix_pts_raw = np.float32([
    [1689, 2074],  # P1: 근거리 (카메라 앞, 아래)
    [3271,  458],  # P2: 오른쪽 위
    [1794,  325],  # P3: 먼 위쪽 중앙
    [  91,  518],  # P4: 왼쪽 위
])

# 이 좌표들을 뽑은 원본 프레임 해상도 (반드시 실제 값으로!)
# 만약 지금 처리(W,H)와 같다면 아래 두 줄을 proc_w/H로 바꿔도 OK
calib_w, calib_h = W, H   # 예: 같은 프레임에서 뽑았다면 이렇게
proc_w,  proc_h  = W, H

if (calib_w != proc_w) or (calib_h != proc_h):
    sx, sy = proc_w / float(calib_w), proc_h / float(calib_h)
    PIX_PTS = pix_pts_raw * np.array([sx, sy], dtype=np.float32)
else:
    PIX_PTS = pix_pts_raw.copy()

# 반코트 실제 좌표(단위 m): 시계방향으로 (0,0)->(10,0)->(10,20)->(0,20)
REAL_PTS = np.float32([
    [0.0, 20.0],   # P1: 근거리 아래
    [10.0, 20.0],  # P2: 오른쪽 위쪽 코너
    [10.0, 0.0],   # P3: 먼 위 (센터 라인쪽)
    [0.0, 0.0],    # P4: 왼쪽 위
])

# 호모그래피 계산
Hmat, _ = cv2.findHomography(PIX_PTS, REAL_PTS, method=0)
assert Hmat is not None and Hmat.shape == (3, 3), "Homography failed: check PIX_PTS / REAL_PTS order."
print("Hmat:\n", Hmat)

def to_court_xy(x, y):
    """픽셀(x,y) -> 코트 평면(m)"""
    if Hmat is None:
        return None
    pt = np.array([[x], [y], [1.0]], dtype=np.float32)  # (3,1)
    w = Hmat @ pt                                      # (3,1)
    w /= (w[2, 0] + 1e-9)
    return float(w[0, 0]), float(w[1, 0])



# ===== [ADD-A] 발 좌표 EMA/헬퍼 & 로깅/궤적 버퍼 =====
import csv

# 플레이어 발 좌표 안정화(EMA)
foot_ema = {}          # {oid: np.array([fx, fy])}
FOOT_KEEP = 0.7        # 0.6~0.8 권장(높을수록 부드럽고 반응성↓)

def foot_point_from_mask(m_bin):
    """
    마스크의 가장 아래(y최대) 라인 근처 x들의 중앙값을 발로 추정.
    m_bin: bool(또는 0/1) 2D
    return (fx, fy) or None
    """
    ys, xs = np.where(m_bin)
    if xs.size == 0:
        return None
    yb = ys.max()
    band = ys >= (yb - 2)         # 바닥선 근방 2px 밴드
    xs_band = xs[band]
    if xs_band.size == 0:
        xs_band = xs[ys == yb]    # 밴드가 비면 최하단 픽셀 전부
    fx = float(np.median(xs_band))
    fy = float(yb)
    return fx, fy

# 좌표 로그 & 궤적 버퍼
coord_logs = []         # 프레임별 좌표/크기/속도 로그
trail = {}              # {oid: deque([(x,y), ...])}
TRAIL_LEN = 30

# ------------------------
# 비디오 루프
# ------------------------
OUT_PATH = "/content/drive/MyDrive/Little_kid_0912/result/pitch_coordinate.mp4"
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

# 안전하게 첫 프레임으로 실제 크기/ FPS 취득
_tmp = cv2.VideoCapture(MP4)
ok, first = _tmp.read()
assert ok, "첫 프레임 읽기 실패. 비디오 경로/코덱을 확인하세요."
H0, W0 = first.shape[:2]                 # numpy는 (행=H, 열=W)
fps = _tmp.get(cv2.CAP_PROP_FPS) or 30.0
_tmp.release()

# 이후 계산에서 쓰는 W,H도 실제 프레임 크기로 덮어쓰기
W, H = int(W0), int(H0)
frame_size = (W, H)

fourcc = cv2.VideoWriter_fourcc(*"mp4v") # 안 되면 "avc1" 또는 .avi + "MJPG"
out = cv2.VideoWriter(OUT_PATH, fourcc, float(fps), frame_size)
if not out.isOpened():
    # 코덱 호환 이슈 대응
    fourcc = cv2.VideoWriter_fourcc(*"avc1")
    out = cv2.VideoWriter(OUT_PATH, fourcc, float(fps), frame_size)
    if not out.isOpened():
        raise RuntimeError("VideoWriter 열기 실패: 코덱(mp4v/avc1) 또는 확장자(mp4/avi)를 바꿔보세요.")

# 실제 읽기용 cap (propagate에서 fi로 프레임 동기화)
cap = cv2.VideoCapture(MP4)

# 동적 리시드 주기를 위한 공 속도 추정 히스토리 ★ NEW
ball_hist = deque(maxlen=8)  # (fi, cx, cy)
def px_per_frame_speed():
    if len(ball_hist) < 2:
        return 0.0
    (f1, x1, y1), (f2, x2, y2) = ball_hist[-2], ball_hist[-1]
    dt = max(1, f2 - f1)
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5 / dt

max_jump_ppf = 50.0  # 프레임당 최대 허용 이동 픽셀(해상도/카메라에 맞게 조정) ★ NEW

# 신규 oid 발급 예산(프레임당) ★ NEW
NEW_OID_BUDGET_MAX = 3

with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    for fi, obj_ids, masks in pred.propagate_in_video(state):
        # 프레임 동기화(안전) ★ NEW
        cap.set(cv2.CAP_PROP_POS_FRAMES, fi)
        ok, bgr = cap.read()
        if not ok: break
        overlay = bgr.copy()

        # ---------------- reseed ----------------
        # 공 속도 기반 동적 주기 ★ NEW
        speed = px_per_frame_speed()
        reseed_interval = 5 if speed > 25 else 15

        if fi % reseed_interval == 0:
            res_p = player_model(bgr, conf=0.3, verbose=False)[0]
            # 공: 낮은 conf / 큰 imgsz / agnostic_nms ★ NEW
            res_b = ball_model(bgr, conf=0.08, iou=0.4, imgsz=IMGSZ_B,
                               agnostic_nms=True, verbose=False)[0]

            new_oid_budget = NEW_OID_BUDGET_MAX  # 프레임당 신규 제한 ★ NEW

            for res, label in [(res_p, "player"), (res_b, "ball")]:
                if res.boxes is None or len(res.boxes) == 0:
                    continue

                # 후보들 순회(공일 때는 conf 높은 순으로 먼저 본다) ★ NEW
                confs = res.boxes.conf.detach().float().cpu().numpy()
                order = np.argsort(-confs)
                xyxys = res.boxes.xyxy.detach().cpu().numpy()

                for idx in order:
                    b = xyxys[idx].tolist()
                    matched = False

                    # 이전 EMA 마스크에서 박스 만들고 IOU & 크기변화율로 동일 객체 판단 ★ NEW
                    for oid in list(class_of.keys()):
                        if class_of[oid] != label:
                            continue
                        prev_mask = ema_masks.get(oid)
                        if prev_mask is None:
                            continue

                        ys, xs = np.where(prev_mask > 0.5)
                        if xs.size == 0:
                            continue

                        box_prev = [xs.min(), ys.min(), xs.max(), ys.max()]

                        # 크기 변화율 검사 ★ NEW
                        def ok_size(prev_box, bb, tol=3.0):
                            pw = max(1.0, prev_box[2]-prev_box[0]); ph = max(1.0, prev_box[3]-prev_box[1])
                            bw = max(1.0, bb[2]-bb[0]);            bh = max(1.0, bb[3]-bb[1])
                            s = (bw*bh)/(pw*ph)
                            return (1/tol) <= s <= tol

                        if iou(box_prev, b) > 0.5 and ok_size(box_prev, b, tol=3.0):
                            # ✅ 같은 객체로 판단 → 박스 + 포인트 프롬프트로 보강
                            x1, y1, x2, y2 = b
                            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
                            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
                            labs = torch.tensor([1],        device=dev, dtype=torch.long)

                            pred.add_new_points_or_box(
                                state, fi, oid,
                                points=pts, labels=labs, box=box_t,
                                normalize_coords=True
                            )
                            matched = True

                            # 공 속도 히스토리 갱신 ★ NEW
                            if label == "ball":
                                ball_hist.append((fi, xc, yc))
                            break

                    if not matched:
                        # 신규 oid 발급 전, 기존 트랙들과 IOU가 너무 작으면 skip ★ NEW
                        if new_oid_budget <= 0:
                            continue

                        same_label_prev_boxes = []
                        for _oid, _lbl in class_of.items():
                            if _lbl != label: continue
                            pm = ema_masks.get(_oid)
                            if pm is None: continue
                            ys, xs = np.where(pm > 0.5)
                            if xs.size == 0: continue
                            same_label_prev_boxes.append([xs.min(), ys.min(), xs.max(), ys.max()])

                        max_prev_iou = 0.0
                        for pb in same_label_prev_boxes:
                            max_prev_iou = max(max_prev_iou, iou(pb, b))

                        if label == "ball":
                            min_iou_for_new = 0.05  # 작은 객체라 낮게
                        else:
                            min_iou_for_new = 0.10

                        if len(same_label_prev_boxes) > 0 and max_prev_iou < min_iou_for_new:
                            continue  # 노이즈 신규 방지

                        new_oid = seed_object(fi, b, label, use_box=True)
                        seed_logs.append((new_oid, label, fi))
                        new_oid_budget -= 1

                        # 공 신규 생성 시 히스토리에 추가 ★ NEW
                        if label == "ball":
                            cx, cy = box_center(b)
                            ball_hist.append((fi, cx, cy))

        # ---------------- 마스크 시각화 ----------------
        seen_ball_this_frame = False  # ★ NEW

        for oid, m in zip(obj_ids, masks):
            oid = int(oid)
            m = m.detach().cpu().numpy().squeeze()
            if m.ndim != 2:
                continue

            m = (m > 0).astype(np.uint8) * 255

            if class_of.get(oid) == "ball":
                # 공은 후처리 약하게(보존) ★ NEW
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((2,2), np.uint8))
                # blur는 생략 또는 아주 약하게
                seen_ball_this_frame = True
            else:
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
                m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
                m = cv2.GaussianBlur(m, (3,3), 0)

            cur = (m > 127).astype(np.float32)
            prev = ema_masks.get(oid)
            keep = 0.35 if class_of.get(oid) == "ball" else 0.7  # 공은 반응성↑ ★ NEW
            ema = cur if prev is None else keep * prev + (1 - keep) * cur
            ema_masks[oid] = ema
            m_bin = ema > 0.5

            col = (0,0,255) if class_of.get(oid) == "ball" else color_for(oid)
            overlay[m_bin] = col
            ys, xs = np.where(m_bin)
            if xs.size:
                x1, x2, y1, y2 = xs.min(), xs.max(), ys.min(), ys.max()
                cv2.rectangle(overlay, (x1, y1), (x2, y2), col, 2)
                cv2.putText(overlay, f"{class_of.get(oid)} {oid}", (x1, max(0, y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)

                # ===== [ADD-B] 플레이어=발(foot) / 공=중심(center) 좌표 산출, 궤적, CSV =====
                # 1) 포인트 선택
                if class_of.get(oid) == "player":
                    foot = foot_point_from_mask(m_bin)
                    if foot is not None:
                        fx, fy = foot
                        # EMA로 부드럽게
                        if oid not in foot_ema:
                            foot_ema[oid] = np.array([fx, fy], dtype=np.float32)
                        else:
                            foot_ema[oid] = FOOT_KEEP * foot_ema[oid] + (1 - FOOT_KEEP) * np.array([fx, fy], dtype=np.float32)
                        px, py = float(foot_ema[oid][0]), float(foot_ema[oid][1])
                        point_type = "foot"
                    else:
                        # 가려질 때: bbox 하단 중앙으로 대체
                        px, py = (x1 + x2) / 2.0, float(y2)
                        point_type = "foot_fallback"
                else:
                    # 공: 마스크(bbox) 중심 유지
                    px, py = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                    point_type = "center"

                # 2) 공 속도용 히스토리 갱신(이미 위에서 갱신하는 곳이 있어도 중복 무해)
                if class_of.get(oid) == "ball":
                    ball_hist.append((fi, px, py))

                # 3) 궤적 라인 그리기
                if oid not in trail:
                    trail[oid] = deque(maxlen=TRAIL_LEN)
                trail[oid].append((int(px), int(py)))
                pts = list(trail[oid])
                for k in range(1, len(pts)):
                    cv2.line(overlay, pts[k-1], pts[k], col, 2)

                # 4) 정규화 좌표/박스크기/속도(공만) 계산
                cx_n, cy_n = px / W, py / H
                bw, bh = (x2 - x1), (y2 - y1)
                bw_n, bh_n = bw / W, bh / H

                speed_pps = None
                if class_of.get(oid) == "ball" and len(ball_hist) >= 2:
                    (pf1, px1, py1), (pf2, px2, py2) = ball_hist[-2], ball_hist[-1]
                    dtf = max(1, pf2 - pf1)
                    ppf = ((px2 - px1)**2 + (py2 - py1)**2) ** 0.5 / dtf
                    speed_pps = ppf * (fps if fps else 30)

                # 5) 포인트 표시 & 라벨 텍스트
                cv2.circle(overlay, (int(px), int(py)), 4, col, -1)
                label_txt = f"{class_of.get(oid)} {oid} ({int(px)},{int(py)})"
                if speed_pps is not None:
                    label_txt += f"  {speed_pps:.1f}px/s"
                cv2.putText(overlay, label_txt, (x1, min(H-5, y2+18)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.55, col, 2)

                court_x, court_y = (None, None)
                xy = to_court_xy(px, py) if Hmat is not None else None
                if xy is not None:
                    court_x, court_y = xy

                # 6) CSV 로깅
                coord_logs.append({
                    "frame": int(fi),
                    "time_s": float(fi / (fps if fps else 30)),
                    "oid": int(oid),
                    "class": class_of.get(oid),
                    "point_type": point_type,            # player=foot/foot_fallback, ball=center
                    "x": float(px), "y": float(py),
                    "x_norm": float(cx_n), "y_norm": float(cy_n),
                    "bbox_w": float(bw), "bbox_h": float(bh),
                    "bbox_w_norm": float(bw_n), "bbox_h_norm": float(bh_n),
                    "speed_px_per_s": None if speed_pps is None else float(speed_pps),
                    "court_x_m": None if court_x is None else float(court_x),
                    "court_y_m": None if court_y is None else float(court_y),
                })


                # 공 중심 히스토리(마스크 기준)도 갱신 ★ NEW
                if class_of.get(oid) == "ball":
                    cx, cy = (x1 + x2)/2.0, (y1 + y2)/2.0
                    ball_hist.append((fi, cx, cy))

        bgr = cv2.addWeighted(overlay, 0.35, bgr, 0.65, 0)
        out.write(bgr)

cap.release(); out.release()

# ===== [ADD-C] CSV 저장 =====
CSV_PATH = "/content/drive/MyDrive/Little_kid_0912/result/pitch_coordinate_log.csv"
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

fields = ["frame","time_s","oid","class","point_type",
          "x","y","x_norm","y_norm",
          "bbox_w","bbox_h","bbox_w_norm","bbox_h_norm",
          "speed_px_per_s","court_x_m","court_y_m"]

with open(CSV_PATH, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for row in coord_logs:
        writer.writerow(row)

print("✅ coords saved:", CSV_PATH)

print("✅ saved:", OUT_PATH)

Obtaining file:///content/drive/MyDrive/sam2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13861 sha256=7cde5c9eb433447458da6e403e26eea15b3c2ca378fb4ba70ad4177af5acd2c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-xad6v8na/wheels/8f/2d/45/d6856ebec9610a653a6f66783921cefb2b908fa4ee04453249
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
초기 씨딩 완료: players=3, ball=1, details= [(0, 'player', 0), (1, 'player'

propagate in video: 100%|██████████| 391/391 [12:53<00:00,  1.98s/it]

✅ coords saved: /content/drive/MyDrive/Little_kid_0912/result/pitch_coordinate_log.csv
✅ saved: /content/drive/MyDrive/Little_kid_0912/result/pitch_coordinate.mp4
